# Lab: SQL and Database

This lab focuses on extracting and displaying information from a database. 

Please run all the code chunks until you see the lab exercises (at the very bottom). 

In [1]:
# import libraries
import sqlalchemy
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import Column, Integer, String, ForeignKey, and_, or_
from sqlalchemy.orm import relationship, sessionmaker
from sqlalchemy import func

In [2]:
# Setting things up:
engine = sqlalchemy.create_engine('sqlite:///geog.db')
Base = declarative_base() 

/tmp/ipykernel_8858/737009441.py:3: MovedIn20Warning: The ``declarative_base()`` function is now available as sqlalchemy.orm.declarative_base(). (deprecated since: 2.0) (Background on SQLAlchemy 2.0 at: https://sqlalche.me/e/b8d9)
  Base = declarative_base()


In [3]:
# Schemas
class Region(Base):
  __tablename__ = 'regions'

  id = Column(Integer, primary_key=True)
  name = Column(String)
  departments = relationship("Department", backref = "region")

  def __init__(self, name):
    self.name = name 

  def __repr__(self):
    return "<Region('%s')>" % self.id 

In [4]:
class Department(Base):
  __tablename__ = 'departments'

  id = Column(Integer, primary_key=True)
  deptname = Column(String)
  region_id = Column(Integer, ForeignKey('regions.id')) 
  towns = relationship("Town", backref = "department")

  def __init__(self, deptname):
    self.deptname = deptname 

  def __repr__(self):
    return "<Department('%s')>" % self.id 

In [5]:
class Town(Base):
  __tablename__ = 'towns'

  id = Column(Integer, primary_key=True)
  name = Column(String)
  population = Column(Integer)
  dept_id = Column(Integer, ForeignKey('departments.id'))

  def __init__(self, name, population):
    self.name = name 
    self.population = population

  def __repr__(self):
    return "<Town('%s')>" % (self.name)

In [6]:
#First time create tables
Base.metadata.create_all(engine) 

In [7]:
#Create a session to actually store things in the db
Session = sessionmaker(bind=engine)
session = Session()

In [8]:
# Create regions
reg1 = Region('Region 1')
reg2 = Region('Region 2')
reg3 = Region('Region 3')

In [9]:
# Create departments, nested in regions
dept1 = Department('Department 1')
reg1.departments.append(dept1)

dept2 = Department('Department 2')
reg1.departments.append(dept2)

dept3 = Department('Department 3')
reg3.departments.append(dept3)

dept4 = Department('Department 4')
reg2.departments.append(dept4)


In [10]:
# Create towns, nested in departments
t1 = Town("a", 10000)
t2 = Town("b", 20000)
dept1.towns = [t1, t2]

t3 = Town("c", 30000)
t4 = Town("d", 40000)
dept2.towns = [t3, t4]

t5 = Town("e", 50000)
t6 = Town("f", 60000)
dept3.towns = [t5, t6]

t7 = Town("g", 70000)
t8 = Town("h", 80000)
dept4.towns = [t7, t8]

In [11]:
session.add_all([reg1, reg2, reg3])
session.add_all([dept1, dept2, dept3, dept4])
session.add_all([t1, t2, t3, t4, t5, t6, t7, t8])

session.commit()

1. Print all of the towns, and order them by Town.id in descending order.

In [46]:
from sqlalchemy import inspect
import pandas as pd

query = "SELECT * FROM main.towns ORDER BY id DESC"

df = pd.read_sql(query, engine)
df

,id,name,population,dept_id
0,8,f,60000,4
1,7,e,50000,4
2,6,h,80000,3
3,5,g,70000,3
4,4,d,40000,2
5,3,c,30000,2
6,2,b,20000,1
7,1,a,10000,1


2. Display, by department, the cities having more than 50,000 inhabitants.

In [52]:
query = "SELECT dept_id FROM main.towns where population > 50000"
df = pd.read_sql(query, engine)
df

,id,name,population,dept_id
0,5,g,70000,3
1,6,h,80000,3
2,8,f,60000,4


3. Display the total number of inhabitants per department using only a query. (no lists!)

In [51]:
query = "SELECT sum(population) FROM main.towns GROUP BY dept_id"
df = pd.read_sql(query, engine)
df

,sum(population)
0,30000
1,70000
2,150000
3,110000
